# 2-photon imaging Demo

This notebook contains exercises split across multiple lessons that span the following primary topics
* conditionals
* lists
* loops
* numpy
* visualization

The exercise modules are centered around analysis of neuroscience 2-photon imaging data, which involves recording __two-dimensional__ (X/Y space) changes in brightness of activity-dependent fluorescent proteins that are expressed in cells of interest. Typically the activity data stream consists of a 3-dimensional dataset (X, Y, and time; i.e. a video). For this particular dataset (acquired by Dr. Vijay Namboodiri), the data were collected through a GRIN lens of GCaMP6s-expressing neurons in orbitofrontal cortex. The data consists of single frames in the format of tifs, where each tif file is a sample collected across time.

The exercises follow a sequence starting from identifying the list of files corresponding to each frame, filtering the list for particular frame numbers, loading the frames into numpy arrays, and visualizing the images. 


In [ ]:
import os
import numpy as np
import glob
import tifffile as tiff

import matplotlib.pyplot as plt

In [ ]:
file_dir = r'sample_data\VJ_OFCVTA_7_260_D6_cropped'
search_path = os.path.join(file_dir,"*.tif")

print(search_path)


The * indicates a "wildcard" that represents any number or type of characters that could be in that specific location in the string

For example the variable `search_path` paired with the glob function below, will search for file paths that start with `file_dir` and end in `.tif`. As long as those two conditions are met, it won't matter what the contents in between are (hence the * wildcard).

Glob is a nice built-in module that finds paths and files based on supplied search parameters. If there are multiple files that satisfy the glob search argument, it will return a list of paths. 

In [ ]:
glob_list = sorted(glob.glob(search_path))
glob_list

In [ ]:
len(glob_list)

## List Part 1, Exercise 1: Maybe we don't want to load in all the files above. What can we do to filter the list?

In [ ]:
glob_first_50 = glob_list[:50]
print(glob_first_50)

glob_last_50 = glob_list[50:]

In [ ]:
# this is how we get every other file starting from the 0th (even files). 
glob_every_other_even = glob_list[::2]
glob_every_other_even

# How do we get every odd file? Print the contents of `glob_every_other` and use the len() function to confirm you filtered the right files

## Lists Part 2, Exercise 1 (Conditional and list combined): Removing items from lists/Removing frames from load lists

In [ ]:
remove_item = 'sample_data\\VJ_OFCVTA_7_260_D6_cropped\\VJ_OFCVTA_7_260_D6_cropped0046.tif'

if remove_item in glob_every_other_even: # exercise goal: plug in the right variables
    print('Removing item')
    glob_every_other_even.remove(remove_item)
else:
    print('Item not present in list!')


In [ ]:
glob_every_other_even[20:30]

## Lists Part 2, Exercise 2 (Lists, conditionals, Loops, and List-comprehension): Filtering lists with list comprehension and removing frames from load list

In [ ]:
for entry in glob_every_other_even:
    if 'cropped000' in entry:
        print(entry)

In [ ]:
filtered_list = []
for entry in glob_every_other_even:
    if 'cropped000' not in entry:
        filtered_list.append(entry) 

filtered_list[:10]

Now try to recreate the above statement using a more pythonic format, list comprehension. It should compress the code in the cell above into one line! assign the output to the `filtered_list_compreh` variable to make sure the output matches above.

Use the following format: [ `x` for `x` in `list` if `filter_item` not in `x` ]

In [ ]:
filtered_list_compreh = [ entry for entry in glob_every_other_even if 'cropped000' not in entry ]
filtered_list_compreh[:10]

## Working with 3D and 2D arrays (numpy)

In [ ]:
first_frame = tiff.imread(glob_every_other_even[0])
first_frame

In [ ]:
first_frame.shape

In [ ]:
plt.imshow(first_frame)

In [ ]:
plt.imshow(tiff.imread(glob_every_other_even[40]))

### Numpy exercise 3: Let's load up all 2p frames from the glob list into a numpy array. 

Using the shape method, figure out what argument you need to pass into the np.empty function in order to initialize the numpy array for populating the frame data.

Then as we loop through each file that's loaded in from the file list, figure out which position the frame_counter needs to be in to properly slot in each loaded frame into the array.

In [ ]:
data_2p = np.empty([len(glob_every_other_even),142,247])
frame_counter = 0
for frame_path in glob_every_other_even:
    data_2p[frame_counter,:,:] = tiff.imread(frame_path)
    frame_counter += 1 # =+ doesn't work!

In [ ]:
plt.imshow(np.mean(data_2p, axis=0))

In [ ]:
data_2p_mean = np.mean(data_2p, axis=0)

data_2p_mean.argmax()

### Visualization Exercise 3: plotting max point on a 2-photon imaging 2D heatmap 

** Hint: Research and use the function `np.unravel_index()`. You'll also need to make use of the method `.shape`

In [ ]:
max_xy_coord = np.unravel_index(data_2p_mean.argmax(), data_2p_mean.shape) 

plt.imshow(np.mean(data_2p, axis=0))
plt.plot(max_xy_coord[1], max_xy_coord[0], marker="o", markersize=10, markerfacecolor="red")

### Visualization Exercise 4: Cropping/slicing data before heatmap plot

Use numpy array slicing to visualize a subset of the imaging FOV

In [ ]:
data_2p_cropped = data_2p[:, 55:80, 50:200]

In [ ]:
plt.imshow(np.mean(data_2p_cropped, axis=0))

We can do the same thing as above, but with posthoc limitations of the plot view. 

In [ ]:
plt.imshow(np.mean(data_2p, axis=0))
plt.xlim([50,200])
plt.ylim([80,55])

Note that if your goal is to save memory and computing power, this strategy is not the ideal. But if you need flexibility and are dealing manageable datasets, this strategy is acceptable.